# The Brochure Maker

## LLM project for recognizing important web pages and summarizing their content into a brochure

We will create a product that builds a brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://dalskoric.com")
ed.links

['https://dalskoric.com/',
 'https://dalskoric.com',
 'https://dalskoric.com/about/',
 'https://dalskoric.com/blog/',
 'https://dalskoric.com/portfolio/',
 'https://dalskoric.com/contact/',
 'https://www.instagram.com/',
 'https://www.facebook.com/',
 'https://x.com/',
 'https://dalskoric.com/about',
 'https://dalskoric.com/ai-language-models-comparison-finding-your-perfect-mach/',
 'https://dalskoric.com/category/use-ai-to-work/',
 'https://dalskoric.com/ai-language-models-comparison-finding-your-perfect-mach/',
 'https://dalskoric.com/machine-learning-task-platforms-your-guide-to-getting-started/',
 'https://dalskoric.com/category/use-ai-to-work/',
 'https://dalskoric.com/machine-learning-task-platforms-your-guide-to-getting-started/',
 'https://dalskoric.com/the-case-of-velvet-sundown-the-most-successful-ai-band/',
 'https://dalskoric.com/category/learn-ml-ai/',
 'https://dalskoric.com/the-case-of-velvet-sundown-the-most-successful-ai-band/',
 'https://dalskoric.com/blog',
 'https:/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://dalskoric.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://dalskoric.com/
https://dalskoric.com
https://dalskoric.com/about/
https://dalskoric.com/blog/
https://dalskoric.com/portfolio/
https://dalskoric.com/contact/
https://www.instagram.com/
https://www.facebook.com/
https://x.com/
https://dalskoric.com/about
https://dalskoric.com/ai-language-models-comparison-finding-your-perfect-mach/
https://dalskoric.com/category/use-ai-to-work/
https://dalskoric.com/ai-language-models-comparison-finding-your-perfect-mach/
https://dalskoric.com/machine-learning-task-platforms-your-guide-to-getting-started/
https://dalskoric.com/category/use-ai-to-work/
https://dalskoric.com/machine-learning-task-platforms-your-guide-to-getting-started/
https://dalskoric.com

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so we are using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-1.5B',
 '/tencent/Hunyuan-MT-7B',
 '/meituan-longcat/LongCat-Flash-Chat',
 '/openbmb/MiniCPM-V-4_5',
 '/tencent/HunyuanWorld-Voyager',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/apple/fastvlm-webgpu',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/bytedance-research/USO',
 '/spaces/multimodalart/Qwen-Image-Edit-Fast',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets/data-agents/jupyter-agent-dataset',
 '/datasets/openai/healthbench',
 '/datasets/facebook/recycling_the_web',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/mic

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
microsoft/VibeVoice-1.5B
Updated
3 days ago
•
173k
•
1.4k
tencent/Hunyuan-MT-7B
Updated
1 day ago
•
2.45k
•
440
meituan-longcat/LongCat-Flash-Chat
Updated
4 days ago
•
15.1k
•
387
openbmb/MiniCPM-V-4_5
Updated
about

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-1.5B\nUpdated\n3 days ago\n•\n173k\n•\n1.4k\ntencent/Hunyuan-MT-7B\nUpdated\n1 day ago\n•\n2.45k\n•\n440\nmeituan-longcat/LongCat-Flash-Chat\nUpdated\n4 days ago\n•\n15.1k\n•\n387\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 10 hours ago\n•\n18.2k\n•\n853\ntencent/HunyuanWorld-Voyager\nUpdated\n18 minutes ago\n•\n187\n•\n342\nBrowse 1M+ models\nSpaces\nRunning\n12.7k\n12.7k\n

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}]}


# Hugging Face Company Brochure

---

## **About Hugging Face**

Welcome to Hugging Face, the AI community shaping the future of machine learning. Our platform enables collaboration on cutting-edge models, datasets, and applications, making it the home of innovative AI development.

### **What We Offer**
- **Models**: Browse and collaborate on over 1 million ML models, including state-of-the-art solutions from leading enterprises and independent creators.
- **Datasets**: Access an extensive library of over 250,000 datasets for various ML tasks, ensuring you have the data to build effective models.
- **Spaces**: Run and manage over 400,000 applications directly in our collaborative spaces.

---

## **Our Culture**

At Hugging Face, we’re committed to creating an inclusive environment that fosters creativity and collaboration. Our community consists of over 50,000 organizations, including top firms like Google, Microsoft, and Amazon. We prioritize open-source contributions and welcome developers, researchers, and enthusiasts to share their work and enhance the ML landscape.

### **Community Engagement**
- **Collaborative Environment**: Contributing to our hub allows individuals and organizations to showcase their innovations and build their portfolios.
- **Open Source Philosophy**: We are dedicated to maintaining transparency and collaboration through our robust set of open-source tools, including Transformers, Diffusers, and Tokenizers.

---

## **Career Opportunities**

Join us in building the future of AI! We are continuously seeking talented individuals who are passionate about machine learning. At Hugging Face, you will have the opportunity to work alongside experts in the field and contribute to groundbreaking projects.

### **Why Work with Us?**
- **Innovative Projects**: Engage in projects that push the boundaries of AI technology.
- **Collaborative Culture**: Work with a diverse team that values creativity and open-mindedness.
- **Professional Growth**: Access resources and mentorship to help you advance in your career.

Explore our career opportunities on our [Jobs Page](https://huggingface.co/jobs).

---

## **Customer Base**

Over 50,000 organizations trust Hugging Face, including notable players in the industry such as:
- **Meta**: 2.22k models
- **Intel**: 243 models
- **Grammarly**: 11 models

Join a community where you can leverage shared knowledge, tools, and resources to accelerate your AI development.

---

Join the machine learning revolution today with Hugging Face—where your ideas and our platform create the future of AI.

For more information, visit our website: [Hugging Face](https://huggingface.co)

--- 

**Hugging Face – The AI community building the future.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## Who We Are
At **Hugging Face**, we are a vibrant community dedicated to building the future of artificial intelligence. Our platform unites machine learning enthusiasts, researchers, and developers to collaborate on **models**, **datasets**, and **applications**. We believe in open collaboration and innovation, making advanced AI accessible to everyone.

## Our Offerings
### Models
Hugging Face hosts over **1 million models** including state-of-the-art algorithms for text, image, video, and audio processing. Notable trending models include:
- **microsoft/VibeVoice-1.5B**
- **tencent/Hunyuan-MT-7B**

### Datasets
With access to over **250,000 datasets**, we facilitate comprehensive research and development in machine learning. Explore shared resources and contribute your own findings.

### Spaces
Our application section includes **400,000+ apps**, like **FastVLM** for real-time video captioning and others that allow users to create innovative solutions effortlessly.

### Enterprise Solutions
We offer tailored **Compute and Enterprise** solutions for organizations wanting to enhance their AI capabilities with enterprise-grade security and dedicated support, starting at just **$20/user/month**.

## Our Community
More than **50,000 organizations**, including industry leaders like Google, Microsoft, and Amazon, have turned to Hugging Face to accelerate their AI ambitions. We foster a culture of collaboration where:
- Members share resources.
- Engage in discussions about the latest in AI.
- Contribute to the evolution of machine learning tools.

## Company Culture
At Hugging Face, we champion a **collaborative** and **inclusive culture**. We focus on:
- **Open Source**: Building on community contributions to advance machine learning tools like Transformers and Diffusers.
- **Innovation**: Encouraging the development of cutting-edge models and datasets.
- **Continuous Learning**: Providing ample resources and platforms for personal and professional growth.

## Careers at Hugging Face
We are always on the lookout for passionate individuals to join our team. Opportunities span across various roles in engineering, research, product management, and community engagement. At Hugging Face, you’ll find:
- A diverse and supportive work environment.
- Opportunities to work on some of the most impactful AI projects.
- The ability to collaborate with experts and innovators in the field.

### Join Us!
Whether you're a potential customer, investor, or recruit, we invite you to be part of the AI revolution at Hugging Face. 

**Explore more at [Hugging Face](https://huggingface.co)**.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'company social page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'company social page', 'url': 'https://twitter.com/huggingface'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


# Hugging Face Brochure

## Overview
**Hugging Face** is an innovative AI community dedicated to building the future of machine learning. Recognized as a powerhouse platform where collaboration thrives, Hugging Face allows users to explore, create, and deploy models, datasets, and applications in a communal atmosphere.

### Our Mission
We strive to provide an open-source platform that empowers the machine learning community to share knowledge, resources, and expertise. At Hugging Face, we believe that collaborative efforts will accelerate advancements in AI and machine learning.

---

## Our Offerings

### Models and Datasets
- **1M+ Models:** Explore an extensive library of machine learning models, featuring contributions from organizations like Microsoft, Google, and Tencent.
- **250k+ Datasets:** Access a diverse array of datasets suitable for various machine learning tasks, supporting research and application development.

### Spaces and Applications
- **Create & Share:** Host unlimited public models, datasets, and applications. Our platform supports numerous AI apps, from video generation to real-time captioning.

### Enterprise Solutions
Hugging Face provides enterprise-grade tools and support to meet the needs of organizations, ensuring high-level security, access controls, and dedicated support systems. With competitive pricing and scalable solutions starting as low as $20/user/month, we cater to over 50,000 organizations globally.

---

## Community Culture
At Hugging Face, we foster an inclusive and collaborative culture that emphasizes knowledge sharing and community engagement. The following principles guide our community:
- **Open Source:** We believe in building the foundation of ML tooling with the community through shared resources and collective growth.
- **Transparency:** Everyone is welcome to share insights through community discussions, articles, and papers, driving collective learning and development.

### Notable Achievements
- Recent contributions to datasets and publication of research papers highlight the vibrant activity within the Hugging Face community, emphasizing our commitment to advancing AI together.

---

## Careers at Hugging Face
Join us in our mission to shape the future of AI! We welcome passionate individuals who thrive in a dynamic and collaborative environment. Opportunities are available across various fields including engineering, research, marketing, and community management.

- **Why Work Here?**
  - Engage with cutting-edge technologies.
  - Collaborate with a world-class team.
  - Contribute to impactful open-source projects.
  - Enjoy a flexible and growth-oriented work environment.

Explore current job openings at [Hugging Face Careers](https://huggingface.co/jobs).

---

## Join Us
Whether you are a developer, researcher, or just passionate about AI, Hugging Face offers the right resources and community for you. Join us today to start contributing and benefiting from collaborative machine learning solutions!

[Visit our website](https://huggingface.co) for more information and to get started.

---

**Hugging Face - Building the future of AI together.**